# This code uses the xarray ffill (forward fill) and bfill (backfill) functions to fill in the NaN values in the input datafiles with neighboring values.

### The horizontal interpolation (horiz_interp) routine used throughout FMS is called by data_overide and takes a field on a lat lon grid and interpolates it according to the method you provide in the dataTable in the xml.

### Horiz_interp did not like the NaN values and caused the model simulation to blow up with "Extreme surface values!!". To get around this issue with the NaN values, I used ffill and bfill in xarray to fill the neighboring values.

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import xesmf as xe
import bottleneck

# plots show in the notebook
%matplotlib inline

# additional configuration (optional)
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina'

In [12]:
ds_taux_fafmip = xr.open_dataset('/net/rlb/fafmip_cm4/fafmip_gfdl_cm4_input/FAFMIP_tauu_v2.nc')
ds_tauy_fafmip = xr.open_dataset('/net/rlb/fafmip_cm4/fafmip_gfdl_cm4_input/FAFMIP_tauv_v2.nc')

ds_taux_fafmip = ds_taux_fafmip.rename_vars({'surface_downward_eastward_stress':'taux_adj'})
ds_tauy_fafmip = ds_tauy_fafmip.rename_vars({'surface_downward_northward_stress':'tauy_adj'})

taux_adj = ds_taux_fafmip['taux_adj']  # These are now DataArrays
tauy_adj = ds_tauy_fafmip['tauy_adj']  # These are now DataArrays

taux_adj, tauy_adj

(<xarray.DataArray 'taux_adj' (time: 12, latitude: 180, longitude: 360)>
 [777600 values with dtype=float32]
 Coordinates:
   * longitude  (longitude) float64 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
   * latitude   (latitude) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
   * time       (time) object 0061-01-16 12:00:00 ... 0061-12-16 12:00:00
 Attributes:
     standard_name:  surface_downward_eastward_stress
     units:          Pa
     submodel:       1
     stash_code:     3219
     long_name:      x-component of windstress anomaly
     cell_methods:   area: mean time: mean,
 <xarray.DataArray 'tauy_adj' (time: 12, latitude: 180, longitude: 360)>
 [777600 values with dtype=float32]
 Coordinates:
   * longitude  (longitude) float64 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
   * latitude   (latitude) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
   * time       (time) object 0061-01-16 12:00:00 ... 0061-12-16 12:00:00
 Attributes:
     standard_name:  surface_downward_

In [25]:
# Use ffill and bfill to fill the NaN values by propagating values forward and backward
# along the latitude dimension.

taux_adj_fill = xr.DataArray(taux_adj.ffill(dim='latitude',limit=None)\
                                     .bfill(dim='latitude',limit=None))
                             
tauy_adj_fill = xr.DataArray(tauy_adj.ffill(dim='latitude',limit=None)\
                                     .bfill(dim='latitude',limit=None))   
    
# Check to make sure the NaN values are gone!!
taux_adj.isel(time=0), taux_adj_fill.isel(time=0), \
tauy_adj.isel(time=0), tauy_adj_fill.isel(time=0)

(<xarray.DataArray 'taux_adj' (latitude: 180, longitude: 360)>
 array([[      nan,       nan,       nan, ...,       nan,       nan,       nan],
        [      nan,       nan,       nan, ...,       nan,       nan,       nan],
        [-0.000744, -0.000661, -0.000584, ..., -0.000561, -0.000662, -0.000747],
        ...,
        [ 0.002048,  0.002066,  0.002073, ...,  0.001938,  0.001977,  0.002018],
        [      nan,       nan,       nan, ...,       nan,       nan,       nan],
        [      nan,       nan,       nan, ...,       nan,       nan,       nan]],
       dtype=float32)
 Coordinates:
   * longitude  (longitude) float64 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
   * latitude   (latitude) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
     time       object 0061-01-16 12:00:00
 Attributes:
     standard_name:  surface_downward_eastward_stress
     units:          Pa
     submodel:       1
     stash_code:     3219
     long_name:      x-component of windstress anomaly
     

In [26]:
# Save output to netcdf file
taux_adj_fill.to_netcdf("taux_fafmip_NaN_values_filled_with_neighboring_values.nc")
tauy_adj_fill.to_netcdf("tauy_fafmip_NaN_values_filled_with_neighboring_values.nc")